In [16]:
import pandas as pd
import numpy as np
def read_binary_file(file_path):
    with open(file_path, 'rb') as file:
        lines = file.read().decode('utf-8').split('\n')
    return lines
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        input_text = file.read()
    return input_text

gujarati_text = read_file('../test_datasets/dev.guj_Gujr')
nepali_text = read_file('../test_datasets/dev.npi_Deva')
burmese_text = read_file('../test_datasets/dev.mya_Mymr')
khmer_text = read_file('../test_datasets/dev.khm_Khmr')
galician_text = read_file('../test_datasets/dev.glg_Latn')
english_labels = read_binary_file('../test_datasets/dev.eng_Latn')
english_labels = [[i] for i in english_labels]

In [17]:
# Correct language codes for M2M100
language_codes = {
    'gujarati': 'gu',
    'nepali': 'ne',
    'burmese': 'my',
    'khmer': 'km',
    'galician': 'gl',
    'english': 'en'
}

# Test sentences in different languages
input_texts = {
    'gujarati': gujarati_text,  
    'nepali': nepali_text,  
    'burmese': burmese_text,  
    'khmer': khmer_text,  
    'galician': galician_text  
}

In [18]:
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the input files
def load_input_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Correct language codes for small-100
language_codes = {
    'gujarati': 'gu',
    'nepali': 'ne',
    'burmese': 'my',
    'khmer': 'km',
    'galician': 'gl',
    'english': 'en'
}

# Load the SMaLL-100 model and tokenizer
model_name = 'alirezamsh/small100'
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Function to chunk text
def chunk_text(text, max_length=128):
    sentences = text.split('\n')
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > max_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += sentence_length

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Function to translate text
def test_translation(source_text, source_lang, target_lang='en', max_length=128):
    input_text = f"{source_text} </s> {target_lang}"
    encoded_text = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    generated_tokens = model.generate(**encoded_text, max_length=max_length)
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translated_text

# Process translations
for lang, text in input_texts.items():
    print(f"Processing translation from {lang} to English...")
    chunks = chunk_text(text)
    translated_chunks = [test_translation(chunk, language_codes[lang]) for chunk in chunks]
    translated_text = '\n'.join(translated_chunks)

    # Save the translated text to a file
    with open(f"{lang}.txt", "w", encoding='utf-8') as file:
        file.write(translated_text)


Processing translation from gujarati to English...


KeyboardInterrupt: 

In [12]:
import sacrebleu

# Load the translated texts
def load_translated_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().split('\n')


# Translated files
translated_files = {
    'burmese': 'burmese.txt',
    'galician': 'galician.txt',
    'gujarati': 'gujarati.txt',
    'khmer': 'khmer.txt',
    'nepali': 'nepali.txt'
}

# Calculate BLEU scores for each translated file
bleu_scores = {}
for lang, file_path in translated_files.items():
    translated_text = load_translated_file(file_path)
    # translated_text = [[i] for i in translated_text]
    bleu = sacrebleu.corpus_bleu(translated_text, english_labels)
    bleu_scores[lang] = bleu.score

# Display the BLEU scores
for lang, score in bleu_scores.items():
    print(f"BLEU score for {lang}: {score}")

BLEU score for burmese: 9.609104620988687
BLEU score for galician: 57.39394461331334
BLEU score for gujarati: 44.19620110203889
BLEU score for khmer: 45.07376830625059
BLEU score for nepali: 56.558866868458445
